In [1]:
from cobra.io import read_sbml_model
from mewpy.simulation import get_simulator
import pandas as pd
import cobra
from mewpy.simulation import set_default_solver
set_default_solver('gurobi')

In [2]:
%store -r t_2020_4 #CCLE 2020 gene expression data formatted for input
expression = t_2020_4.transpose()
%store -r media_only_essential_reactions #calculated in separate notebook using essential_reactions functions from mewpy.simulation package
%store -r media_only_essential_genes #calculated in separate notebook using essential_genes functions from mewpy.simulation package
%store -r RPMI1640_plus_2mM_glutamine #media conditions defined as dictionary in a separate notebook
media = RPMI1640_plus_2mM_glutamine
indexes = list(range(0, 64))
for n in range(len((t_2020_4.index).to_list())):
    if 'IGROV1' == ((t_2020_4.index).to_list())[n]: #iterate through gene expression dataframe to find target cell line
        t = n
row_number_transcriptomics = t
print('row:',t)
experimental_growth = 0.0376 #manually set experimental growth rate threshold
                            #growth rate == 1/(doubling time in hours)

row: 41


In [4]:
#load Human1 model and solve FBA optimised for biomass production, with media conditions only as constraint
model = read_sbml_model('Human-GEM-annotated.xml')
media_only_simulation = get_simulator(model, envcond = media)
media_only_result = media_only_simulation.simulate()
media_only_result.find(['biomass_human'])

Set parameter Username
Academic license - for non-commercial use only - expires 2024-03-24


,Flux rate
Reaction ID,
biomass_human,0.055515


In [5]:
#create dictionary containing expression values
#for non-Human models, 'ENS' term in the generateion of g_ensembl list must be changed to be organism-specific
expression = expression.transpose()
t = expression
gene_exp = t.iloc[row_number_transcriptomics,:]

g_list = []
for k in gene_exp.keys():
    g_list.append(k)
    
g_list = []
for k in gene_exp.keys():
    g_list.append(k)

g_ensembl = []
for n in range(len(g_list)):
    if 'ENS' not in (g_list[n].split())[0]:
        g_ensembl.append((g_list[n].split())[1])
    if 'ENS' in (g_list[n].split())[0]:
        g_ensembl.append((g_list[n].split())[0])
        
punc = '''!()-[]{};:'"\,<>./?@#$%^&*_~'''
g_ensembl_2 = []
for n in range(len(g_ensembl)):
    for x in g_ensembl[n]:
        if x in punc:
            g_ensembl[n] = g_ensembl[n].replace(x, "")
    g_ensembl_2.append(g_ensembl[n])
g_ensembl_2

vs = []
for v in gene_exp:
    vs.append(v)
    
gene_exp_dict = {}
for n in range(len(g_ensembl_2)):
    gene_exp_dict[g_ensembl_2[n]] = vs[n]
    
ANDs = []
ORs = []
ANDORs = []
one_gene = []
no_gene = []

for r in model.reactions:
    if 'and' in r.gene_reaction_rule and 'or' not in r.gene_reaction_rule:
        ANDs.append(r.id)
    if 'and' in r.gene_reaction_rule and 'or' in r.gene_reaction_rule:
        ANDORs.append(r.id)
    if 'or' in r.gene_reaction_rule and 'and' not in r.gene_reaction_rule:
        ORs.append(r.id)
    if len(r.gene_reaction_rule) == 0:
        no_gene.append(r.id)
    if len(r.gene_reaction_rule) != 0:
        if 'or' in r.gene_reaction_rule:
            continue
        elif 'and' in r.gene_reaction_rule:
            continue
        else:
            one_gene.append(r.id)

print('AND rules: ', len(ANDs))
print('ANDOR rules: ', len(ANDORs))
print('OR rules: ', len(ORs))
print('ONE GENE rules: ', len(one_gene))
print('NO GENE rules: ', len(no_gene))
print('Proportion of model not annotated: ', len(no_gene)/len(model.reactions)*100, '%')
print('Proportion of model which IS annotated: ', 100-(len(no_gene)/len(model.reactions)*100), '%')
print('Total Reactions = ', len(model.reactions))
print(len(ORs)+len(ANDs)+len(ANDORs)+len(one_gene)+len(no_gene))

one_gene_forward = []
one_gene_reversible = []
for r in one_gene:
    if model.reactions.get_by_id(r).reversibility == True:
        one_gene_reversible.append(r)
    else:
        one_gene_forward.append(r)
print('number of reversible, one gene reactions =', len(one_gene_reversible))
print('number of forward, one gene reactions =', len(one_gene_forward))

AND rules:  653
ANDOR rules:  129
OR rules:  3972
ONE GENE rules:  3282
NO GENE rules:  5060
Proportion of model not annotated:  38.63775198533904 %
Proportion of model which IS annotated:  61.36224801466096 %
Total Reactions =  13096
13096
number of reversible, one gene reactions = 1074
number of forward, one gene reactions = 2208


In [8]:
one_gene_constrained_reactions = {} #create dictionary to contain reaction bounds, according to code below
                                    #this dictionary will be used by MEWpy to constrain model 
for r in one_gene_forward:
    if r not in media.keys(): #do not overwrite media conditions
        if r not in media_only_essential_reactions: #do not constrain essential reactions/genes, or growth will be blocked
            if model.reactions.get_by_id(r).gene_reaction_rule not in media_only_essential_genes:
                if model.reactions.get_by_id(r).gene_reaction_rule in gene_exp_dict.keys(): #check we have expression data
                    if gene_exp_dict[model.reactions.get_by_id(r).gene_reaction_rule] != (0,0):
                        model.reactions.get_by_id(r).bounds = (0,(gene_exp_dict[model.reactions.get_by_id(r).gene_reaction_rule])) #set upper bound equal to expression value
                        solution = model.optimize() #optimise after constraint, using COBRApy
                        if solution.fluxes['biomass_human'] < experimental_growth: 
                            print(r, ':', 'constrained bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                            model.reactions.get_by_id(r).bounds = (0,1000)
                            solution = model.optimize()
                            print(r, ':', 're-opened bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                            one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds #save reaction bounds to dictionary
                        if solution.fluxes['biomass_human'] >= experimental_growth:
                            print(r, ':', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                            one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
                            
for r in one_gene_reversible: #run constraints code for reversible, one-gene reactions
    if r not in media.keys():
        if r not in media_only_essential_reactions: #change to new dict
            if model.reactions.get_by_id(r).gene_reaction_rule not in media_only_essential_reactions:
                if model.reactions.get_by_id(r).gene_reaction_rule in gene_exp_dict.keys(): 
                    if gene_exp_dict[model.reactions.get_by_id(r).gene_reaction_rule] != (0,0): 
                        #here, set lower bound as -1*expression value, and upper bound as positive expression value
                        model.reactions.get_by_id(r).bounds = (-1*(gene_exp_dict[model.reactions.get_by_id(r).gene_reaction_rule]),(gene_exp_dict[model.reactions.get_by_id(r).gene_reaction_rule]))
                        solution = model.optimize()
                        if solution.fluxes['biomass_human'] < experimental_growth:
                            print(r, ':', 'constrained bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                            model.reactions.get_by_id(r).bounds = (-1000,1000)
                            solution = model.optimize()
                            print(r, ':', 're-opened bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                            #update constraints dictionary with reversible reaction bounds
                            one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
                        if solution.fluxes['biomass_human'] >= experimental_growth:
                            print(r, ':', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                            one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
                            
or_forward = []
or_reversible = []
for r in ORs:
    if model.reactions.get_by_id(r).reversibility == True:
        or_reversible.append(r)
    else:
        or_forward.append(r)
print('number of reversible, OR reactions =', len(or_reversible))
print('number of forward, OR reactions =', len(or_forward))

for r in or_forward:
    if r not in media.keys():
        if r not in media_only_essential_reactions:
            rule_list = [] #make list to append with terms in reaction rule
            rule = model.reactions.get_by_id(r).gene_reaction_rule
            rule_list.append(rule.split())
            rule_genes = [] #append new list with only ensembl IDs from reaction rule
            for n in rule_list:
                for num in n:
                    if 'ENS' in num:
                        rule_genes.append(num)
            #calculate sum of expressions
            genes_in_dataset = []
            for gene in rule_genes:
                if gene in gene_exp_dict.keys():
                    genes_in_dataset.append(gene)
            exp_list = []        
            for g in genes_in_dataset:
                exp_list.append(gene_exp_dict[g])
            sum_of_expressions = sum(exp_list)
            #set bounds according to sum of expressions
            model.reactions.get_by_id(r).bounds = (0,sum_of_expressions) #same as 'one-gene' method, where expression (sum) is upper bound
            solution = model.optimize()
            if solution.fluxes['biomass_human'] < experimental_growth:
                print(r, ':', 'constrained bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                model.reactions.get_by_id(r).bounds = (0,1000)
                solution = model.optimize()
                print(r, ':', 're-opened bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
            if solution.fluxes['biomass_human'] >= experimental_growth:
                print(r, ':', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
                
for r in or_reversible:
    if r not in media.keys():
        if r not in media_only_essential_reactions:
            #find ensembl ids in gene-reaction rules
            rule_list = []
            rule = model.reactions.get_by_id(r).gene_reaction_rule
            rule_list.append(rule.split())
            rule_genes = []
            for n in rule_list:
                for num in n:
                    if 'ENS' in num:
                        rule_genes.append(num)
            #calculate sum of expressions
            genes_in_dataset = []
            for gene in rule_genes:
                if gene in gene_exp_dict.keys():
                    genes_in_dataset.append(gene)
            exp_list = []        
            for g in genes_in_dataset:
                exp_list.append(gene_exp_dict[g])
            sum_of_expressions = sum(exp_list)
            #set bounds according to sum of expressions; same as before, use negative and positive expression (sum) as bounds
            model.reactions.get_by_id(r).bounds = (-1*sum_of_expressions,sum_of_expressions)
            solution = model.optimize()
            if solution.fluxes['biomass_human'] < experimental_growth:
                print(r, ':', 'constrained bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                model.reactions.get_by_id(r).bounds = (-1000,1000)
                solution = model.optimize()
                print(r, ':', 're-opened bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
            if solution.fluxes['biomass_human'] >= experimental_growth:
                print(r, ':', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
                
and_forward = []
and_reversible = []
for r in ANDs:
    if model.reactions.get_by_id(r).reversibility == True:
        and_reversible.append(r)
    else:
        and_forward.append(r)
print('number of reversible, AND reactions =', len(and_reversible))
print('number of forward, AND reactions =', len(and_forward))

for r in and_forward:
    if r not in media.keys():
        if r not in media_only_essential_reactions:
            #find ensembl IDs in reaction rules, as performed for 'or' rules
            rule_list = []
            rule = model.reactions.get_by_id(r).gene_reaction_rule
            rule_list.append(rule.split())
            rule_genes = []
            for n in rule_list:
                for num in n:
                    if 'ENS' in num:
                        rule_genes.append(num)
            #generate list of expressions
            genes_in_dataset = []
            for gene in rule_genes:
                if gene in gene_exp_dict.keys():
                    genes_in_dataset.append(gene)
            exp_list = []        
            for g in genes_in_dataset:
                exp_list.append(gene_exp_dict[g])
            if len(exp_list) > 1:
                exp_list.sort()
                min_expression = exp_list[0]
            if len(exp_list) == 1:
                min_expression = exp_list[0] #set variable for minimum, rate-limiting subunit expression
            #use minimum expression as upper bound for forward reactions
            model.reactions.get_by_id(r).bounds = (0,min_expression)
            solution = model.optimize()
            if solution.fluxes['biomass_human'] < experimental_growth:
                print(r, ':', 'constrained bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                model.reactions.get_by_id(r).bounds = (0,1000)
                solution = model.optimize()
                print(r, ':', 're-opened bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
            if solution.fluxes['biomass_human'] >= experimental_growth:
                print(r, ':', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
                
for r in and_reversible:
    if r not in media.keys():
        if r not in media_only_essential_reactions:
            rule_list = []
            rule = model.reactions.get_by_id(r).gene_reaction_rule
            rule_list.append(rule.split())
            rule_genes = []
            for n in rule_list:
                for num in n:
                    if 'ENS' in num:
                        rule_genes.append(num)
            #generate list of expressions
            genes_in_dataset = []
            for gene in rule_genes:
                if gene in gene_exp_dict.keys():
                    genes_in_dataset.append(gene)
            exp_list = []        
            for g in genes_in_dataset:
                exp_list.append(gene_exp_dict[g])
            if len(exp_list) > 1:
                exp_list.sort()
                min_expression = exp_list[0]
            if len(exp_list) == 1:
                min_expression = exp_list[0]
            #set bounds according to minimum expression; use negative and positive expression (minimum) as bounds
            model.reactions.get_by_id(r).bounds = ((-1*min_expression),min_expression)
            solution = model.optimize()
            if solution.fluxes['biomass_human'] < experimental_growth:
                print(r, ':', 'constrained bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                model.reactions.get_by_id(r).bounds = (-1000,1000)
                solution = model.optimize()
                print(r, ':', 're-opened bounds', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
            if solution.fluxes['biomass_human'] >= experimental_growth:
                print(r, ':', model.reactions.get_by_id(r).bounds, solution.fluxes['biomass_human'])
                one_gene_constrained_reactions[r] = model.reactions.get_by_id(r).bounds
                
all_and_simulation = get_simulator(model, envcond = media, constraints = one_gene_constrained_reactions)
all_and_result = all_and_simulation.simulate()
print('growth with media AND gene expression constraints: ', all_and_result.find(['biomass_human']).iloc[0,0],
      'g/gDW/hour','=',(1/all_and_result.find(['biomass_human']).iloc[0,0]),'hours')

HMR_3907 : (0, 6.55581615506164) 0.055514528776719314
HMR_4097 : (0, 4.403267722339301) 0.055514528776719314
HMR_4108 : (0, 4.403267722339301) 0.055514528776719314
HMR_4133 : (0, 4.403267722339301) 0.055514528776719314
HMR_4360 : (0, 2.1342209397606338) 0.055514528776719314
HMR_4372 : (0, 4.083213368248981) 0.055514528776719314
HMR_7747 : (0, 5.497931651896279) 0.055514528776719314
HMR_8360 : (0, 5.772413554748482) 0.055514528776719314
HMR_8757 : (0, 0.2016338611696504) 0.055514528776719314
HMR_5397 : (0, 3.8308637567517576) 0.055514528776719314
HMR_5399 : (0, 2.899175630480513) 0.055514528776719314
HMR_5400 : (0, 2.899175630480513) 0.055514528776719314
HMR_8592 : (0, 4.308156975114762) 0.055514528776719314
HMR_8589 : (0, 4.308156975114762) 0.055514528776719314
HMR_8583 : (0, 0.0) 0.055514528776719314
HMR_8584 : (0, 0.0) 0.055514528776719314
HMR_8585 : (0, 1.4168397419128294) 0.055514528776719314
HMR_3944 : (0, 4.929317594911761) 0.055514528776719314
HMR_4132 : (0, 4.238022517825721) 0

HMR_8439 : (0, 0.3219280948873623) 0.05551452877671932
HMR_8440 : (0, 5.801934646273351) 0.05551452877671932
HMR_8441 : (0, 1.4168397419128294) 0.05551452877671932
HMR_9486 : (0, 2.3305584000308026) 0.05551452877671932
HMR_4429 : (0, 0.5360529002402097) 0.05551452877671932
HMR_4437 : (0, 0.2750070474998698) 0.05551452877671932
HMR_4660 : (0, 0.01435529297707) 0.05551452877671932
HMR_4712 : (0, 0.01435529297707) 0.05551452877671932
HMR_4288 : (0, 1.510961919277379) 0.05551452877671932
HMR_4668 : (0, 1.510961919277379) 0.05551452877671932
HMR_4737 : (0, 5.801934646273351) 0.05551452877671932
HMR_4739 : (0, 5.801934646273351) 0.05551452877671932
HMR_6415 : (0, 5.992088609013996) 0.05551452877671932
HMR_6985 : (0, 4.85598969730848) 0.05551452877671932
HMR_8021 : (0, 0.0976107966264223) 0.05551452877671932
HMR_8091 : (0, 0.1505596765753814) 0.05551452877671932
HMR_4681 : (0, 0.1505596765753814) 0.05551452877671932
HMR_4703 : (0, 0.5849625007211562) 0.05551452877671932
HMR_4704 : (0, 0.58496

HMR_7329 : (0, 3.266036893995317) 0.05551452877671932
HMR_7332 : (0, 4.130107178658833) 0.05551452877671932
HMR_7429 : (0, 3.683696454306516) 0.05551452877671932
HMR_7574 : (0, 4.165911938935689) 0.05551452877671932
HMR_7575 : (0, 3.647314509547136) 0.05551452877671932
HMR_7585 : (0, 4.165911938935689) 0.05551452877671932
HMR_7586 : (0, 3.647314509547136) 0.05551452877671932
HMR_8691 : (0, 4.05484847699562) 0.05551452877671932
HMR_8694 : (0, 3.93451650158608) 0.05551452877671932
HMR_5151 : (0, 0.0) 0.05551452877671937
HMR_5152 : (0, 0.4436066514756146) 0.05551452877671937
HMR_5153 : (0, 0.1890338243900171) 0.05551452877671937
HMR_5154 : (0, 0.0) 0.05551452877671953
HMR_5157 : (0, 5.79051141189419) 0.05551452877671953
HMR_5158 : (0, 0.1763227726404629) 0.05551452877671953
HMR_5159 : (0, 0.0) 0.05551452877671953
HMR_5160 : (0, 0.0) 0.055514528776719675
HMR_5162 : (0, 0.6415460290875237) 0.055514528776719675
HMR_5163 : (0, 1.918386234446348) 0.055514528776719675
HMR_5164 : (0, 7.558650301

HMR_1039 : (0, 0.2986583155645151) 0.05551452877671936
HMR_1062 : (0, 0.2986583155645151) 0.05551452877671936
HMR_1063 : (0, 0.2986583155645151) 0.05551452877671936
HMR_1064 : (0, 0.2986583155645151) 0.05551452877671936
HMR_1073 : (0, 0.2986583155645151) 0.05551452877671936
HMR_1075 : (0, 0.137503523749935) 0.05551452877671936
HMR_1077 : (0, 0.137503523749935) 0.05551452877671936
HMR_1079 : (0, 3.493134922305505) 0.05551452877671936
HMR_1081 : (0, 0.2141248053528473) 0.05551452877671936
HMR_8545 : (0, 0.0) 0.05551452877671936
HMR_8550 : (0, 0.2141248053528473) 0.05551452877671936
HMR_8554 : (0, 0.01435529297707) 0.05551452877671936
HMR_8555 : (0, 0.0) 0.05551452877671936
HMR_8556 : (0, 0.1110313123887439) 0.05551452877671936
HMR_3475 : (0, 3.765534746362977) 0.05551452877671936
HMR_2825 : (0, 2.6915341649192) 0.05551452877671936
HMR_2829 : (0, 2.6915341649192) 0.05551452877671936
HMR_2830 : (0, 3.475084882948783) 0.05551452877671936
HMR_2833 : (0, 2.6915341649192) 0.05551452877671936
H

HMR_8233 : (0, 3.424922088210688) 0.055514528776719314
HMR_8235 : (0, 3.424922088210688) 0.055514528776719314
HMR_8237 : (0, 4.428946344769483) 0.055514528776719314
HMR_8238 : (0, 4.428946344769483) 0.055514528776719314
HMR_8242 : (0, 0.6870606883398922) 0.055514528776719314
HMR_8248 : (0, 5.787118982813345) 0.055514528776719314
HMR_8249 : (0, 0.1243281350022017) 0.055514528776719314
HMR_8250 : (0, 0.1243281350022017) 0.055514528776719314
HMR_8251 : (0, 0.1243281350022017) 0.055514528776719314
HMR_0673 : (0, 4.071247819461443) 0.055514528776719314
HMR_0674 : (0, 4.071247819461443) 0.055514528776719314
HMR_0675 : (0, 4.071247819461443) 0.055514528776719314
HMR_0676 : (0, 4.071247819461443) 0.055514528776719314
HMR_0677 : (0, 4.071247819461443) 0.05551452877671934
HMR_0678 : (0, 4.071247819461443) 0.05551452877671934
HMR_4296 : (0, 0.0285691521967709) 0.05551452877671934
HMR_0478 : (0, 0.5360529002402097) 0.05551452877671934
HMR_0483 : (0, 3.795974694206668) 0.05551452877671934
HMR_0482 

...

TDCA3St : (-0.411426245726465, 0.411426245726465) 0.05551452877671813
THYOCHOLt : (-2.5043102114049494, 2.5043102114049494) 0.05551452877671813
TUDCA3St : (-0.411426245726465, 0.411426245726465) 0.05551452877671813
UCHOLt : (-2.5043102114049494, 2.5043102114049494) 0.05551452877671813
UDCA3St : (-0.411426245726465, 0.411426245726465) 0.05551452877671813
UDCHOLt : (-2.5043102114049494, 2.5043102114049494) 0.05551452877671813
2HATVACIDthc : (-0.070389327891398, 0.070389327891398) 0.05551452877671813
2HATVLACthc : (-0.070389327891398, 0.070389327891398) 0.05551452877671813
35DHPVSthc : (-0.070389327891398, 0.070389327891398) 0.05551452877671813
3HPVSthc : (-0.070389327891398, 0.070389327891398) 0.05551452877671813
3ISPVSthc : (-0.070389327891398, 0.070389327891398) 0.05551452877671813
4HATVACIDthc : (-0.070389327891398, 0.070389327891398) 0.05551452877671813
4HATVLACthc : (-0.070389327891398, 0.070389327891398) 0.05551452877671813
6EPVSthc : (-0.070389327891398, 0.070389327891398) 0.05551

HMR_3347 : (0, 1.3950627995175775) 0.04423707218540039
HMR_3348 : (0, 1.3950627995175775) 0.04423707218540039
HMR_3349 : (0, 4.725195817226664) 0.04423707218540039
HMR_3351 : (0, 1.3950627995175775) 0.04423707218540039
HMR_3352 : (0, 1.3950627995175775) 0.04423707218540039
HMR_3353 : (0, 4.725195817226664) 0.04423707218540039
HMR_3365 : (0, 1.3950627995175775) 0.04423707218540039
HMR_3367 : (0, 1.3950627995175775) 0.04423707218540039
HMR_3368 : (0, 4.725195817226664) 0.04423707218540039
HMR_3370 : (0, 1.3950627995175775) 0.04423707218540039
HMR_3372 : (0, 1.3950627995175775) 0.04423707218540039
HMR_3373 : (0, 4.725195817226664) 0.04423707218540039
HMR_3501 : (0, 1.3950627995175775) 0.04423707218540039
HMR_3503 : (0, 1.3950627995175775) 0.04423707218540039
HMR_3505 : (0, 4.725195817226664) 0.04423707218540039
HMR_3508 : (0, 1.3950627995175775) 0.04423707218540039
HMR_3509 : (0, 1.3950627995175775) 0.04423707218540039
HMR_3510 : (0, 4.725195817226664) 0.04423707218540039
HMR_3513 : (0, 1

HMR_3533 : (0, 4.339137384919585) 0.04423707218540039
HMR_3534 : (0, 5.792334805706692) 0.04423707218540039
HMR_1644 : (0, 1.3950627995175775) 0.04423707218540039
HMR_0786 : (0, 0.0426443374084937) 0.04423707218540039
HMR_0819 : (0, 0.0426443374084937) 0.04423707218540039
HMR_0830 : (0, 0.0426443374084937) 0.04423707218540039
HMR_0832 : (0, 0.0426443374084937) 0.04423707218540039
HMR_8383 : (0, 2.150559676575381) 0.04423707218540039
HMR_8402 : (0, 4.669593751188333) 0.04423707218540039
HMR_8404 : (0, 4.669593751188333) 0.04423707218540039
HMR_8406 : (0, 4.669593751188333) 0.04423707218540039
HMR_8407 : (0, 4.669593751188333) 0.04423707218540039
HMR_7208 : (0, 5.619999829821357) 0.04423707218540039
HMR_7210 : (0, 5.619999829821357) 0.04423707218540039
HMR_7212 : (0, 5.619999829821357) 0.04423707218540039
HMR_7214 : (0, 5.619999829821357) 0.04423707218540039
HMR_7526 : (0, 4.41886457739032) 0.04423707218540039
HMR_7530 : (0, 4.41886457739032) 0.04423707218540039
HMR_7542 : (0, 4.41886457

FAOXC101C8x : (0, 1.3950627995175775) 0.044237072185429833
FAOXC102C81m : (0, 4.339137384919585) 0.044237072185429833
FAOXC102C81x : (0, 1.3950627995175775) 0.044237072185429833
FAOXC10C10OHm : (0, 4.356143810225276) 0.044237072185429833
FAOXC10C8m : (0, 4.339137384919585) 0.044237072185429833
FAOXC10C8x : (0, 1.3950627995175775) 0.044237072185429833
FAOXC10DCC8DCx : (0, 1.3950627995175775) 0.044237072185429833
FAOXC11BRC9BRx : (0, 1.3950627995175775) 0.044237072185429833
FAOXC11C9m : (0, 4.339137384919585) 0.044237072185429833
FAOXC121C101m : (0, 4.339137384919585) 0.044237072185429833
FAOXC121C10x : (0, 1.3950627995175775) 0.044237072185429833
FAOXC122C101m : (0, 4.339137384919585) 0.044237072185429833
FAOXC123C102m : (0, 4.339137384919585) 0.044237072185429833
FAOXC123C102x : (0, 1.3950627995175775) 0.044237072185429833
FAOXC12C10m : (0, 4.339137384919585) 0.044237072185429833
FAOXC12C10x : (0, 1.3950627995175775) 0.044237072185429833
FAOXC12C12OHm : (0, 4.356143810225276) 0.0442370

HMR_3454 : (-4.339137384919585, 4.339137384919585) 0.04285826367458483
HMR_3455 : (-5.792334805706692, 5.792334805706692) 0.04285826367458483
HMR_3468 : (-1.3950627995175775, 1.3950627995175775) 0.04285826367458483
HMR_3469 : (-4.725195817226664, 4.725195817226664) 0.04285826367458483
HMR_1298 : (-1.3950627995175775, 1.3950627995175775) 0.04285826367458483
HMR_3355 : (-1.3950627995175775, 1.3950627995175775) 0.04285826367458483
HMR_3375 : (-1.3950627995175775, 1.3950627995175775) 0.04285826367458483
HMR_3323 : (-1.3950627995175775, 1.3950627995175775) 0.04285826367458483
HMR_1219 : (-1.3950627995175775, 1.3950627995175775) 0.04285826367458483
HMR_1220 : (-4.339137384919585, 4.339137384919585) 0.04285826367458483
HMR_1221 : (-5.792334805706692, 5.792334805706692) 0.04285826367458483
HMR_1651 : (-4.339137384919585, 4.339137384919585) 0.04285826367458483
HMR_4433 : (-0.0976107966264223, 0.0976107966264223) 0.04285826367458483
HMR_4434 : (-0.0976107966264223, 0.0976107966264223) 0.04285826

In [9]:
#store dataframe of fluxes for comparison to other models
#may aslo want to store copy of model with constraints for future use to avoid running above code
igrov1_t_2020_thresolds = all_and_result.dataframe
%store igrov1_t_2020_thresolds

Stored 'igrov1_t_2020_thresolds' (DataFrame)
